In [1]:
import pandas as pd
import numpy as np

In [2]:
fightersData = pd.read_csv('../fightersDatab.csv')

flyweightw = pd.read_csv('../Weightclass/flyweight.csv')
bantamw = pd.read_csv('../Weightclass/bantamweight.csv')
featherw = pd.read_csv('../Weightclass/featherweight.csv')
lightw = pd.read_csv('../Weightclass/lightweight.csv')
middlew = pd.read_csv('../Weightclass/middleweight.csv')
welterw = pd.read_csv('../Weightclass/welterweight.csv')
lightheavyw = pd.read_csv('../Weightclass/lightheavyweight.csv')
heavyw = pd.read_csv('../Weightclass/heavyweight.csv')

wbantamw = pd.read_csv('../Weightclass/womensbantamweight.csv')
wfeatherw = pd.read_csv('../Weightclass/womensfeatherweight.csv')
wflyw = pd.read_csv('../Weightclass/womensflyweight.csv')
wstraww = pd.read_csv('../Weightclass/womensstrawweight.csv')

divisions = [
    "flyweightw",
    "bantamw",
    "featherw",
    "lightw",
    "middlew",
    "welterw",
    "lightheavyw",
    "heavyw",
    "wbantamw",
    "wfeatherw",
    "wflyw",
    "wstraww"
]

divisionsdf = {
    "flyweightw": flyweightw,
    "bantamw": bantamw,
    "featherw": featherw,
    "lightw": lightw,
    "middlew": middlew,
    "welterw": welterw,
    "lightheavyw": lightheavyw,
    "heavyw": heavyw,
    "wbantamw": wbantamw,
    "wfeatherw": wfeatherw,
    "wflyw": wflyw,
    "wstraww": wstraww
}


In [3]:
fighters = pd.DataFrame()

In [4]:
def calulatingskill(divisionV, currentfighter, skillcolumn):
    a = currentfighter[skillcolumn].values[0]
    mean = divisionV[skillcolumn].mean()
    std = divisionV[skillcolumn].std()
    z = (a - mean) / std
    # x = np.clip(2.5 + z, 0, 5) 
    x = np.clip(50 + z * 10, 0, 100)
    return round(x, 2)

def scale_to_5(value, col_values):
    min_val = min(col_values)
    max_val = max(col_values)
    if max_val == min_val:
        return 50
    # scaled = (value - min_val) / (max_val - min_val) * 5
    scaled = (value - min_val) / (max_val - min_val) * 100
    return round(scaled, 2)

def calculate_enhanced_career_score(row):
    """Enhanced career scoring that properly weights achievements"""
    
    # Base win rate component (0-2 points)
    if row['total_ufc_fights'] > 0:
        win_rate = row['won_fight'] / row['total_ufc_fights']
        # Minimum fight threshold - fighters with very few fights get penalized
        fight_factor = min(row['total_ufc_fights'] / 15, 1.0)  # Reaches full value at 15 fights
        # base_score = win_rate * 2 * fight_factor
        base_score = win_rate * 40 * fight_factor 
    else:
        base_score = 0
    
    # Undefeated bonus (0-0.5 points)
    undefeated_bonus = 0
    if row['loss_fights'] == 0 and row['total_ufc_fights'] >= 5:
        # undefeated_bonus = 0.5
        undefeated_bonus = 10
        
    # Title achievement bonus (0-1.5 points)
    title_bonus = 0
    if row['won_title_fight'] > 0:
        # title_bonus = 0.8 + (row['won_title_fight'] - 1) * 0.2  # 0.8 for first title, +0.2 per defense
        # title_bonus = min(title_bonus, 1.5)
        title_bonus = 16 + (row['won_title_fight'] - 1) * 4  # Changed from 0.8 + * 0.2
        title_bonus = min(title_bonus, 30)
    elif row['title_fight'] > 0:
        title_bonus = 6
    
    
    # Division ranking bonus (0-0.8 points)
    div_rank_bonus = 0
    div_rank = pd.to_numeric(row['div_rank'], errors='coerce')
    if pd.notna(div_rank):
        if div_rank == 0:
            # div_rank_bonus = 0.8
            div_rank_bonus = 16
        elif div_rank <= 3:
            # div_rank_bonus = 0.6
            div_rank_bonus = 12
        elif div_rank <= 5:
            # div_rank_bonus = 0.4
            div_rank_bonus = 8
        elif div_rank <= 10:
            # div_rank_bonus = 0.2 
            div_rank_bonus = 4
    
    # P4P ranking 
    p4p_bonus = 0
    pfp_rank = pd.to_numeric(row['pfp_rank'], errors='coerce')
    if pd.notna(pfp_rank) and pfp_rank > 0:
        if pfp_rank <= 5:
            # p4p_bonus = 0.5
            p4p_bonus = 10
        elif pfp_rank <= 10:
            # p4p_bonus = 0.3
            p4p_bonus = 6
        elif pfp_rank <= 15:
            # p4p_bonus = 0.1
            p4p_bonus = 2
    
    # Volume bonus for active/long careers (0-0.2 points)
    # volume_bonus = min(row['total_ufc_fights'] / 25, 0.2)
    volume_bonus = min(row['total_ufc_fights'] / 25, 4)
    
    total_career_score = base_score + undefeated_bonus + title_bonus + div_rank_bonus + p4p_bonus + volume_bonus
    
    # Scale to 0-5 range
    # career_score = min(total_career_score, 5.0)
    career_score = min(total_career_score, 100.0)
    
    return round(career_score, 2)

def calculate_enhanced_finishing(row):
    """Enhanced finishing calculation that considers context"""
    
    if row['won_fight'] == 0:
        return 0
    
    # Base finishing rate
    finish_rate = (row['ko_t'] + row['totsub']) / row['won_fight']
    
    # Title fight finishing bonus
    title_finish_bonus = 0
    if row['title_fight'] > 0:
        # Assume if they have title fights and high finish rate, they finished in big moments
        if finish_rate > 0.4:  # If they finish 40%+ of fights
            # title_finish_bonus = 0.3
            title_finish_bonus = 6
    
    # Scale finishing rate to 0-4 range, leave room for bonuses
    # base_finish_score = min(finish_rate * 4, 4.0)
    base_finish_score = min(finish_rate * 80, 80.0) 
    
    # Decision rate (going to decision isn't bad, just different style)
    decision_rate = row['totdec'] / row['won_fight']
    # decision_score = decision_rate * 1  # Max 1 point for decision wins
    decision_score = decision_rate * 20
    
    total_finishing = base_finish_score + title_finish_bonus + decision_score
    
    # return round(min(total_finishing, 5.0), 2)
    return round(min(total_finishing, 100.0), 2)


for div in divisions:
    fighterlist = []
    currentdiv = divisionsdf[div]
    
    for x, row in currentdiv.iterrows():
        current_fighter = currentdiv[currentdiv['name'] == row['name']]
        
        # Striking metrics
        splm = calulatingskill(currentdiv, current_fighter, "splm")
        sigatt = calulatingskill(currentdiv, current_fighter, "sig_str_atmpted")
        stracc = calulatingskill(currentdiv, current_fighter, "str_acc")
        kd = calulatingskill(currentdiv, current_fighter, "kd")
        
        # Wrestling metrics
        tdl = calulatingskill(currentdiv, current_fighter, "td_avg")
        tatt = calulatingskill(currentdiv, current_fighter, "td_atmpted")
        tdavg = calulatingskill(currentdiv, current_fighter, "td_avg_acc")
        ctrl = calulatingskill(currentdiv, current_fighter, "ctrl")
        dominance = 0
        if row['total_rounds'] > 0:
            dominance = round((row['ctrl'] / row['total_rounds']) * 100, 2)
        
        # Grappling metrics
        subatt = calulatingskill(currentdiv, current_fighter, "sub_att")
        subavg = calulatingskill(currentdiv, current_fighter, "sub_avg")
        tsub = calulatingskill(currentdiv, current_fighter, "totsub")
        sublosses = 100 - calulatingskill(currentdiv, current_fighter, "totsub_losses")
        
        # Defense metrics
        strdef = calulatingskill(currentdiv, current_fighter, "str_def")
        strabs = 100 - calulatingskill(currentdiv, current_fighter, "sapm")
        takdef = calulatingskill(currentdiv, current_fighter, "td_def")
        
        # Enhanced durability calculation
        if row['loss_fights'] == 0:
            dur = 100
        else:
            # Penalize finishes more than decision losses
            finish_losses = row['ko_losses'] + row['totsub_losses']
            decision_losses = row['totdec_losses']
            
            # Finish losses hurt more than decision losses
            durability_penalty = ((finish_losses * 1.5) + (decision_losses * 0.8)) / row['total_ufc_fights']
            dur = round(100 - (durability_penalty * 100), 2)
            dur = max(dur, 10)
        
        # Use enhanced career and finishing calculations
        career_score = calculate_enhanced_career_score(row)
        finishing = calculate_enhanced_finishing(row)
        
        # Calculate category scores with refined weights
        striking = round((splm * 0.25 + stracc * 0.35 + kd * 0.30 + sigatt * 0.10), 2)
        wrestling = round((tdl * 0.25 + tdavg * 0.35 + ctrl * 0.30 + tatt * 0.10), 2)  
        grappling = round((tsub * 0.40 + subavg * 0.25 + subatt * 0.15 + sublosses * 0.20), 2)
        defense = round((strdef * 0.30 + dur * 0.40 + takdef * 0.20 + strabs * 0.10), 2)
        
        # Handle NaN values
        striking = 0 if pd.isna(striking) else striking
        wrestling = 0 if pd.isna(wrestling) else wrestling  
        grappling = 0 if pd.isna(grappling) else grappling
        defense = 0 if pd.isna(defense) else defense
        finishing = 0 if pd.isna(finishing) else finishing
        
        # Enhanced overall calculation with adjusted weights
        overall = round((
            striking * 0.18 +
            wrestling * 0.22 +
            grappling * 0.13 +
            defense * 0.18 +
            finishing * 0.07 +
            career_score * 0.22
        ), 2)
        
        fighterlist.append({
            'name': row['name'],
            'nickname': row['nick_name'],
            'stance': row['stance'],
            'overall': overall,
            'striking': striking,
            'wrestling': wrestling,
            'grappling': grappling,
            'defense': defense,
            'career_score': career_score,
            'finishing': finishing,
            'dominance': dominance,
            
            'wins': row['wins'],
            'losses': row['losses'],
            'draws': row['draws'],
            'ufcwins': row['won_fight'],
            'ufclosses': row['loss_fights'],
            'ufcdraws': row['total_ufc_fights'] - row['won_fight'] - row['loss_fights'] -  row['nocon_losses'],
            'ufcnocon': row['nocon_losses'],
            'ufctotalfights': row['total_ufc_fights'],
            'Titlefighs_wins': row['won_title_fight'],
            
            'pfp_rank': row['pfp_rank'],
            'div_rank': row['div_rank'],
            'divisions': row['division'],
            'maindiv': row['main_div'],
            
            
            'dob': row['dob'],
            
            'strikes_landed_permin': splm,
            'sigstrikes_attempted': sigatt,
            'striking_acc': stracc,
            'kd_permin': kd,
            'striking_def': strdef,
            'strabsorbpm': strabs,
            
            'takedown_landed': tdl,
            'takedown_attempted': tatt,
            'takedown_acc': tdavg,
            'control_time': ctrl,
            'takedown_def': takdef,
            
            'sub_attempted': subatt,
            'sub_avg15': subavg,
            'total_sub': tsub,
            'sub_resistance': sublosses,
            
            'durability': dur,
        })
        
    
    fighters = pd.DataFrame(fighterlist)
    fighters.to_csv(f"../SkillLevel/{div}.csv", index=False)

In [5]:
fighters

,name,nickname,stance,overall,striking,wrestling,grappling,defense,career_score,finishing,...,takedown_landed,takedown_attempted,takedown_acc,control_time,takedown_def,sub_attempted,sub_avg15,total_sub,sub_resistance,durability
0,Aisling Daly,Ais the Bash,Orthodox,43.88,49.82,52.48,54.26,59.66,9.45,50.00,...,57.79,46.32,50.86,51.99,39.56,48.43,57.33,53.15,56.99,73.33
1,Aleksandra Albu,Stitch,Orthodox,39.28,51.96,51.51,49.81,41.15,5.49,50.00,...,58.91,46.32,54.44,43.65,58.59,44.62,49.43,53.15,47.48,42.50
2,Alex Chambers,Astro Girl,Southpaw,38.73,50.81,51.24,49.52,31.37,2.87,80.00,...,59.29,48.42,50.86,45.92,48.21,52.24,58.91,53.15,28.48,10.00
3,Alexia Thainara,Burguesinha,Orthodox,41.28,49.63,59.39,0.00,72.69,2.71,80.00,...,66.38,41.70,74.37,41.97,25.28,44.62,NaN,53.15,56.99,100.00
4,Alice Ardelean,NaN,Orthodox,31.88,54.84,48.42,0.00,51.93,2.79,20.00,...,47.05,43.80,54.44,44.07,62.92,44.62,NaN,43.41,56.99,46.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Virna Jandiroba,Carcara,Orthodox,54.26,45.24,60.25,68.46,65.08,39.77,50.00,...,59.85,67.74,47.67,72.75,56.86,67.47,71.54,72.65,56.99,78.18
98,Viviane Pereira,Sucuri,Orthodox,37.12,39.78,53.20,47.02,52.97,5.49,20.00,...,40.05,41.70,74.37,43.28,37.39,44.62,46.27,43.41,56.99,60.00
99,Yan Xiaonan,NaN,Orthodox,51.73,57.61,53.12,46.23,61.79,48.52,26.67,...,45.56,47.58,63.61,49.04,52.97,44.62,43.11,43.41,56.99,70.00
100,Yazmin Jauregui,NaN,Orthodox,18.81,0.00,38.38,44.33,0.00,8.20,40.00,...,37.71,43.38,34.52,41.77,68.55,44.62,43.11,43.41,47.48,40.00


In [6]:
flyweightw = pd.read_csv('../SkillLevel/flyweightw.csv')
bantamw = pd.read_csv('../SkillLevel/bantamw.csv')
featherw = pd.read_csv('../SkillLevel/featherw.csv')
lightw = pd.read_csv('../SkillLevel/lightw.csv')
middlew = pd.read_csv('../SkillLevel/middlew.csv')
welterw = pd.read_csv('../SkillLevel/welterw.csv')
lightheavyw = pd.read_csv('../SkillLevel/lightheavyw.csv')
heavyw = pd.read_csv('../SkillLevel/heavyw.csv')

wbantamw = pd.read_csv('../SkillLevel/wbantamw.csv')
wfeatherw = pd.read_csv('../SkillLevel/wbantamw.csv')
wflyw = pd.read_csv('../SkillLevel/wflyw.csv')
wstraww = pd.read_csv('../SkillLevel/wstraww.csv')

divisionsdf = {
    "flyweightw": flyweightw,
    "bantamw": bantamw,
    "featherw": featherw,
    "lightw": lightw,
    "middlew": middlew,
    "welterw": welterw,
    "lightheavyw": lightheavyw,
    "heavyw": heavyw,
    "wbantamw": wbantamw,
    "wfeatherw": wfeatherw,
    "wflyw": wflyw,
    "wstraww": wstraww
}


In [7]:
pd.set_option('display.max_columns', None)
display(flyweightw[flyweightw['name'] == 'Demetrious Johnson'])


,name,nickname,stance,overall,striking,wrestling,grappling,defense,career_score,finishing,dominance,wins,losses,draws,ufcwins,ufclosses,ufcdraws,ufcnocon,ufctotalfights,Titlefighs_wins,pfp_rank,div_rank,divisions,maindiv,dob,strikes_landed_permin,sigstrikes_attempted,striking_acc,kd_permin,striking_def,strabsorbpm,takedown_landed,takedown_attempted,takedown_acc,control_time,takedown_def,sub_attempted,sub_avg15,total_sub,sub_resistance,durability
33,Demetrious Johnson,Mighty Mouse,Orthodox,72.51,58.13,74.64,70.45,71.55,90.05,54.0,7492.68,27,3,1,15,3.0,0.0,0.0,18,12,1.0,0.0,"['flyweight', 'bantamweight']",flyweight,1986/08/13,51.29,57.67,62.21,59.21,66.56,62.71,59.87,90.93,61.14,97.27,53.24,83.61,46.53,87.25,56.89,86.67


In [8]:
flyweightw['strikes_landed_permin'].min()

31.31

In [9]:
flyweightw['strikes_landed_permin'].max()

92.55

In [10]:
min_val = flyweightw['strikes_landed_permin'].min()
max_val = flyweightw['strikes_landed_permin'].max()
print(max_val)
print((51.29 - min_val) )
print((max_val - min_val) )
scaled = ((51.29 - min_val) / (max_val - min_val)) * 100
print(scaled)
# 31.31 51.29 92.55

92.55
19.98
61.239999999999995
32.62573481384716


In [11]:
for division_name, division_data in divisionsdf.items():
    
    # Scale the main category scores to 0-100 within division
    striking_values = division_data['striking'].values
    wrestling_values = division_data['wrestling'].values
    grappling_values = division_data['grappling'].values
    defense_values = division_data['defense'].values
    career_score_values = division_data['career_score'].values
    finishing_values = division_data['finishing'].values
    dominance_values = division_data['dominance'].values
    
    division_data['striking'] = division_data['striking'].apply(lambda x: scale_to_5(x, striking_values))
    division_data['wrestling'] = division_data['wrestling'].apply(lambda x: scale_to_5(x, wrestling_values))
    division_data['grappling'] = division_data['grappling'].apply(lambda x: scale_to_5(x, grappling_values))
    division_data['defense'] = division_data['defense'].apply(lambda x: scale_to_5(x, defense_values))
    division_data['career_score'] = division_data['career_score'].apply(lambda x: scale_to_5(x, career_score_values))
    division_data['finishing'] = division_data['finishing'].apply(lambda x: scale_to_5(x, finishing_values))
    division_data['dominance'] = division_data['dominance'].apply(lambda x: scale_to_5(x, dominance_values))
    
    for _, row in division_data.iterrows():
        striking_components = ['strikes_landed_permin', 'sigstrikes_attempted', 'striking_acc', 'kd_permin']
        current_striking_avg = sum(row[comp] for comp in striking_components) / len(striking_components)
        if current_striking_avg > 0:  # Avoid division by zero
            adjustment_factor = row['striking'] / current_striking_avg
            for comp in striking_components:
                division_data.loc[row.name, comp] = round(min(100, max(0, row[comp] * adjustment_factor)), 2)
        
        wrestling_components = ['takedown_landed', 'takedown_attempted', 'takedown_acc', 'control_time', 'takedown_def']
        current_wrestling_avg = sum(row[comp] for comp in wrestling_components) / len(wrestling_components)
        if current_wrestling_avg > 0:
            adjustment_factor = row['wrestling'] / current_wrestling_avg
            for comp in wrestling_components:
                division_data.loc[row.name, comp] = round(min(100, max(0, row[comp] * adjustment_factor)), 2)
        
        grappling_components = ['sub_attempted', 'sub_avg15', 'total_sub', 'sub_resistance']
        current_grappling_avg = sum(row[comp] for comp in grappling_components) / len(grappling_components)
        if current_grappling_avg > 0:
            adjustment_factor = row['grappling'] / current_grappling_avg
            for comp in grappling_components:
                division_data.loc[row.name, comp] = round(min(100, max(0, row[comp] * adjustment_factor)), 2)
        
        defense_components = ['striking_def', 'strabsorbpm', 'takedown_def', 'durability']
        current_defense_avg = sum(row[comp] for comp in defense_components) / len(defense_components)
        if current_defense_avg > 0:
            adjustment_factor = row['defense'] / current_defense_avg
            for comp in defense_components:
                division_data.loc[row.name, comp] = round(min(100, max(0, row[comp] * adjustment_factor)), 2)
        

    division_data['overall'] = division_data.apply(lambda row: round((
        row['striking'] * 0.18 +
        row['wrestling'] * 0.22 +
        row['grappling'] * 0.13 +
        row['defense'] * 0.18 +
        row['finishing'] * 0.07 +
        row['career_score'] * 0.22
    ), 2), axis=1)
    
    division_data['rank'] = division_data['overall'].rank(method='min', ascending=False).astype(int)
    
    rank_values = division_data['rank']
    division_data = division_data.drop('rank', axis=1)
    division_data.insert(division_data.columns.get_loc('stance') + 1, 'rank', rank_values)
    
    fighters = division_data.sort_values('rank')
    
    fighters.to_csv(f"../SkillLevel/{division_name}.csv", index=False)

In [12]:
flyweightw = pd.read_csv('../SkillLevel/flyweightw.csv')
bantamw = pd.read_csv('../SkillLevel/bantamw.csv')
featherw = pd.read_csv('../SkillLevel/featherw.csv')
lightw = pd.read_csv('../SkillLevel/lightw.csv')
middlew = pd.read_csv('../SkillLevel/middlew.csv')
welterw = pd.read_csv('../SkillLevel/welterw.csv')
lightheavyw = pd.read_csv('../SkillLevel/lightheavyw.csv')
heavyw = pd.read_csv('../SkillLevel/heavyw.csv')

wbantamw = pd.read_csv('../SkillLevel/wbantamw.csv')
wfeatherw = pd.read_csv('../SkillLevel/wbantamw.csv')
wflyw = pd.read_csv('../SkillLevel/wflyw.csv')
wstraww = pd.read_csv('../SkillLevel/wstraww.csv')


In [13]:
flyweightw['striking'].max()

100.0